In [5]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r'E:\invasive-aquatic-species-data\Invasive_Life_Stage'

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for folder in tqdm(os.listdir(directory_path)):
    #print(class_folder)
    clsp = os.path.join(directory_path,folder)  
    for class_folder in os.listdir(clsp):
        #print(class_folder)
        class_path = os.path.join(clsp, class_folder)    
        #print(class_path)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):
                image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)
            
            # Append image and label to lists
                X_train.append(img_array)
                Y_train.append(class_label)
                #print(class_label)
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


100%|█████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.50s/it]

X_train shape: (49227, 32, 32, 3)
Y_train shape: (49227,)


In [6]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
Y_train_encoded = to_categorical(Y_train)
indices = np.arange(len(X_train))
np.random.shuffle(indices)
X_train_shuffled = X_train[indices]
Y_train_encoded_shuffled = Y_train_encoded[indices]
X_train, X_test, y_train, y_test = train_test_split(X_train_shuffled, Y_train_encoded_shuffled, test_size=0.2, random_state=42)

In [3]:
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D

model_path = r"C:\Users\shaif\Downloads\invssl.h5"
model = load_model(model_path)
#model.layers[0].trainable = False
x = model.layers[-8].output  
output = Dense(5, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=40, mode='max', verbose=1)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = new_model.fit(X_train, y_train, batch_size=32, epochs=50,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
985/985 [==============================] - 77s 60ms/step - loss: 30.4342 - accuracy: 0.4101 - val_loss: 4.9776 - val_accuracy: 0.5510
Epoch 2/50
985/985 [==============================] - 52s 53ms/step - loss: 3.8331 - accuracy: 0.4736 - val_loss: 2.5572 - val_accuracy: 0.5865
Epoch 3/50
985/985 [==============================] - 55s 56ms/step - loss: 2.0222 - accuracy: 0.5281 - val_loss: 1.0872 - val_accuracy: 0.5800
Epoch 4/50
985/985 [==============================] - 56s 57ms/step - loss: 1.5799 - accuracy: 0.5510 - val_loss: 1.1352 - val_accuracy: 0.5911
Epoch 5/50
985/985 [==============================] - 59s 60ms/step - loss: 1.1786 - accuracy: 0.5808 - val_loss: 1.0781 - val_accuracy: 0.5522
Epoch 6/50
985/985 [==============================] - 56s 57ms/step - loss: 1.2305 - accuracy: 0.5895 - val_loss: 1.0635 - val_accuracy: 0.6048
Epoch 7/50
985/985 [==============================] - 61s 62ms/step - loss: 1.2049 - accuracy: 0.6165 - val_loss: 1.0110 - val_accuracy

In [ ]:
history = new_model.fit(X_train, y_train, batch_size=32, epochs=2,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
985/985 [==============================] - 59s 60ms/step - loss: 0.9954 - accuracy: 0.7572 - val_loss: 610.0840 - val_accuracy: 0.7565
Epoch 2/50
985/985 [==============================] - 64s 65ms/step - loss: 0.7025 - accuracy: 0.7658 - val_loss: 73.8275 - val_accuracy: 0.7629
Epoch 3/50
985/985 [==============================] - 62s 63ms/step - loss: 0.7918 - accuracy: 0.7715 - val_loss: 14644.0547 - val_accuracy: 0.5303
Epoch 4/50
985/985 [==============================] - 60s 61ms/step - loss: 0.9408 - accuracy: 0.7575 - val_loss: 476.9403 - val_accuracy: 0.7543
Epoch 5/50
985/985 [==============================] - 55s 56ms/step - loss: 0.6378 - accuracy: 0.7722 - val_loss: 444.2742 - val_accuracy: 0.7479
Epoch 6/50
985/985 [==============================] - 58s 59ms/step - loss: 0.6438 - accuracy: 0.7767 - val_loss: 288.0295 - val_accuracy: 0.7673
Epoch 7/50
985/985 [==============================] - 53s 54ms/step - loss: 1.5090 - accuracy: 0.7093 - val_loss: 2223.5852

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

Test Balanced Accuracy: 0.567010311131761
Test Accuracy: 0.6581352833638026


In [ ]:
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(32,32,3), pooling="avg")
x =  Dense(256, activation='relu')(base_model.output)
x = layers.Dropout(0.5)(x)
output = Dense(5, activation='softmax')(x)
ne_model = Model(inputs=base_model.input, outputs=output)
#early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, mode='max', verbose=1)
ne_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = ne_model.fit(X_train, y_train, batch_size=32, epochs=100,validation_split=0.2)

Epoch 1/100
985/985 [==============================] - 72s 66ms/step - loss: 0.7261 - accuracy: 0.7525 - val_loss: 0.5859 - val_accuracy: 0.7861
Epoch 2/100
985/985 [==============================] - 52s 53ms/step - loss: 0.4784 - accuracy: 0.8278 - val_loss: 0.4496 - val_accuracy: 0.8337
Epoch 3/100
985/985 [==============================] - 60s 61ms/step - loss: 0.7421 - accuracy: 0.7315 - val_loss: 0.7625 - val_accuracy: 0.7183
Epoch 4/100
985/985 [==============================] - 55s 56ms/step - loss: 0.6062 - accuracy: 0.7819 - val_loss: 0.5745 - val_accuracy: 0.7888
Epoch 5/100
985/985 [==============================] - 57s 58ms/step - loss: 0.4977 - accuracy: 0.8219 - val_loss: 1.2642 - val_accuracy: 0.5159
Epoch 6/100
985/985 [==============================] - 53s 54ms/step - loss: 0.4630 - accuracy: 0.8355 - val_loss: 1.1537 - val_accuracy: 0.6454
Epoch 7/100
985/985 [==============================] - 57s 57ms/step - loss: 0.4822 - accuracy: 0.8293 - val_loss: 0.9577 - val_ac

985/985 [==============================] - 61s 62ms/step - loss: 0.0844 - accuracy: 0.9711 - val_loss: 0.6260 - val_accuracy: 0.8324
Epoch 58/100
985/985 [==============================] - 61s 62ms/step - loss: 0.0752 - accuracy: 0.9726 - val_loss: 0.5124 - val_accuracy: 0.8814
Epoch 59/100
985/985 [==============================] - 63s 64ms/step - loss: 0.0765 - accuracy: 0.9729 - val_loss: 1.0988 - val_accuracy: 0.8357
Epoch 60/100
985/985 [==============================] - 59s 60ms/step - loss: 0.0804 - accuracy: 0.9723 - val_loss: 2.5837 - val_accuracy: 0.6508
Epoch 61/100
985/985 [==============================] - 58s 59ms/step - loss: 0.0640 - accuracy: 0.9777 - val_loss: 1.8021 - val_accuracy: 0.7569
Epoch 62/100
985/985 [==============================] - 60s 60ms/step - loss: 0.0612 - accuracy: 0.9790 - val_loss: 1.3040 - val_accuracy: 0.8044
Epoch 63/100
985/985 [==============================] - 65s 66ms/step - loss: 0.0644 - accuracy: 0.9778 - val_loss: 0.5215 - val_accuracy

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = ne_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x) 
new_model = Model(inputs=base_model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, Model

# Load the pre-trained SimCLR model
simclr_model = load_model(r"C:\Users\shaif\Downloads\simclr_model.h5")

# Extract the input and output from one branch of the SimCLR model
base_input = simclr_model.input[0]  # Input from the first branch
base_output = simclr_model.get_layer("global_average_pooling2d").output  # Output before projection head of the first branch

# Create a new model using the extracted backbone
backbone_model = Model(inputs=base_input, outputs=base_output)


In [ ]:
def create_classification_model(backbone_model, num_classes=5):
    # Add a classification head
    x = layers.Dense(256, activation='relu')(backbone_model.output)
    x = layers.Dropout(0.5)(x)
    classification_output = layers.Dense(5, activation='softmax')(x)

    # Build the new model
    classification_model = Model(inputs=backbone_model.input, outputs=classification_output)

    return classification_model

# Create the classification model
classification_model = create_classification_model(backbone_model)
#classification_model.summary()
from tensorflow.keras.utils import to_categorical
YT = to_categorical(Y_train)
# Compile the classification model
classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model using the labeled dataset
classification_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
985/985 [==============================] - 31s 25ms/step - loss: 0.8943 - accuracy: 0.7117 - val_loss: 0.5249 - val_accuracy: 0.8083
Epoch 2/100
985/985 [==============================] - 23s 23ms/step - loss: 0.5649 - accuracy: 0.7873 - val_loss: 0.4739 - val_accuracy: 0.8213
Epoch 3/100
985/985 [==============================] - 23s 24ms/step - loss: 0.5083 - accuracy: 0.8064 - val_loss: 0.4587 - val_accuracy: 0.8244
Epoch 4/100
985/985 [==============================] - 27s 27ms/step - loss: 0.4827 - accuracy: 0.8124 - val_loss: 0.4391 - val_accuracy: 0.8361
Epoch 5/100
985/985 [==============================] - 29s 30ms/step - loss: 0.4587 - accuracy: 0.8236 - val_loss: 0.4271 - val_accuracy: 0.8352
Epoch 6/100
985/985 [==============================] - 30s 31ms/step - loss: 0.4386 - accuracy: 0.8326 - val_loss: 0.4218 - val_accuracy: 0.8374
Epoch 7/100
985/985 [==============================] - 34s 35ms/step - loss: 0.4226 - accuracy: 0.8369 - val_loss: 0.4133 - val_ac

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = classification_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)